In [29]:
import zarr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [30]:
def load_single_data(i):
    data = zarr.open('.\dataset\CarFollowing/trainHH.zarr', mode='a')
    start, end = data.index_range[i]

    size_lead = 4.85              # this is for AV
    size_lead = data.lead_size[i] # this is for HV
    size_follow = data.follow_size[i]

    # get timestamps
    timestamps = data.timestamp[start:end]
    # get position, speed, and acceleration
    
    x_lead = data.lead_centroid[start:end]
    v_lead = data.lead_velocity[start:end]
    a_lead = data.lead_acceleration[start:end]
    id=np.ones((1,x_lead.shape[0]))*i
    x_follow = data.follow_centroid[start:end]
    v_follow = data.follow_velocity[start:end]
    a_follow = data.follow_acceleration[start:end]
    array = np.vstack((id, x_lead, v_lead, a_lead, x_follow, v_follow, a_follow,timestamps))
    return array.T

def load_multi_data(i):
    for x in range(i+1):
        single_data = load_single_data(x)
        if x == 0:
            multi_data = single_data
        else:
            multi_data = np.vstack([multi_data, single_data])
    return multi_data
np.savetxt( "site.csv", load_multi_data(1000), delimiter="," )

: 

In [28]:
data=pd.read_csv('site.csv')
data.head()

,0.000000000000000000e+00,0.000000000000000000e+00.1,4.183219729128531128e+00,1.946269610757445978e+00,-1.271794164429400809e+01,1.396879813881541477e+00,2.857759383157767452e-01
0,0.0,0.545917,4.416340,1.944384,-12.731980,1.535397,0.336690
1,0.0,1.062604,4.568766,1.943686,-12.603732,1.372667,0.395086
2,0.0,1.607011,4.753522,1.944021,-12.466701,1.306147,0.693090
3,0.0,2.177451,4.953015,1.945415,-12.303733,1.337162,1.143289
4,0.0,2.774419,5.151196,1.947731,-12.096568,1.483107,1.667380
